# Bitcoin Risk 

## Training Models on pre 2024 data

In [18]:
# get btc daily data
from logic.get import read_data

data = read_data("BTC", "1D")
data.ffill(inplace=True)
data

# create features
from logic.features import log_close_sma_extension

processed_data = log_close_sma_extension(data)
processed_data

# Train risk models using pre 2020 data
from datetime import datetime
import re

from logic.normal import normal_distribution_model

train_X = processed_data.loc[processed_data.DateTime < datetime(2024, 1, 1)].copy(deep=True)

regex = r"^log_[A-Z]{2,3}[0-9]{1,3}_extension$"
log_ma_extension_features = [f for f in train_X.columns if re.match(regex, f)]
model_dictionary = {}

for log_ma in log_ma_extension_features:
    regex =  r"[A-Z]{2,3}[0-9]{1,3}"
    _match = re.search(regex, log_ma)
    #_data[f"risk_{ma}"] = risk_metric(_data[log_ma])
    model_dictionary[log_ma] = normal_distribution_model(train_X[log_ma])

print(model_dictionary)

# Save models as pickles
import pickle

with open("BTC Risk Models", "wb") as handle:
    pickle.dump(model_dictionary, handle, protocol = pickle.HIGHEST_PROTOCOL)

{'log_MA100_extension': NormalDistributionModel(model=<scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7f980d5ed4b0>, mu=0.44150933437107576, var=0.47488241153789595), 'log_MA50_extension': NormalDistributionModel(model=<scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7f980d804460>, mu=0.30930996419145135, var=0.413728593540701), 'log_EMA21_extension': NormalDistributionModel(model=<scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7f980d804ca0>, mu=0.11823431432705257, var=0.12532238065550633), 'log_MA20_extension': NormalDistributionModel(model=<scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7f980d7f8fa0>, mu=0.13766942034416876, var=0.17724488870430927), 'log_MA8_extension': NormalDistributionModel(model=<scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7f980d7f8a00>, mu=0.06088027590894481, var=0.06409398015891925)}


/home/brad/workspace/autotrader/logic/normal.py:28: RuntimeWarning: divide by zero encountered in log
  - (n / 2) * np.log(var)
/home/brad/workspace/autotrader/logic/normal.py:29: RuntimeWarning: divide by zero encountered in scalar divide
  - (1 / (2 * var) * sum((x - mu) ** 2))
/home/brad/workspace/autotrader/logic/normal.py:27: RuntimeWarning: invalid value encountered in scalar subtract
  -(n / 2) * np.log(2 * np.pi)


## Running New Data Through Existing Models

In [19]:
import pickle

with open("BTC Risk Models", "rb") as handle:
    _model_dictionary = pickle.load(handle)

# get btc weekly data 2020 onwards
_data = read_data("BTC", "1W")
_processed_data = log_close_sma_extension(_data)
test_X = _processed_data.iloc[-1,:].copy(deep=True)

# calculate risk
import numpy as np


for log_ma in log_ma_extension_features:
    regex =  r"[A-Z]{2,3}[0-9]{1,3}"
    _match = re.search(regex, log_ma)
    ma = _match.group(0)
    x = test_X[log_ma]
    pdf_data = _model_dictionary[log_ma].model.pdf(x)
    normalised_pdf_data = pdf_data / _model_dictionary[log_ma].model.pdf(_model_dictionary[log_ma].mu)
    factor = np.empty_like(np.array(x))
    if x > _model_dictionary[log_ma].mu:
        factor = 1
    else:
        factor = -1
    test_X[f"risk_{ma}"] = 0.5 + (factor * (1 - normalised_pdf_data) * 0.5)

test_X["combined_risk"] = (
        test_X["risk_EMA21"] 
        + test_X["risk_MA50"] 
        + test_X["risk_MA20"]
        + test_X["risk_MA8"]
    ) / 4

test_X

DateTime               2024-01-22 00:00:00
open                              41583.43
high                              42220.44
low                                38524.0
close                             41877.92
Volume                       138866.037795
Volume MA                    139454.358936
MA100                           28029.0784
MA50                            31047.2492
EMA21                         37410.452921
MA20                            36348.6315
MA8                              42448.335
RSI                              67.071971
RSI MA                           73.396673
ATR1                               3696.44
ATR14                          3924.658571
log_MA100_extension               0.401516
log_MA50_extension                0.299249
log_EMA21_extension               0.112809
log_MA20_extension                0.141602
log_MA8_extension                -0.013529
risk_MA100                         0.49823
risk_MA50                         0.499852
risk_EMA21 

# Ethereum Risk
## Training Models on pre 2024 data

In [28]:
# get eth daily data
from logic.get import read_data

data = read_data("ETH", "1W")
data.ffill(inplace=True)
data

# create features
from logic.features import log_close_sma_extension

processed_data = log_close_sma_extension(data)
processed_data

# Train risk models using pre 2020 data
from datetime import datetime
import re

from logic.normal import normal_distribution_model

train_X = processed_data.loc[processed_data.DateTime < datetime(2024, 1, 1)].copy(deep=True)

regex = r"^log_[A-Z]{2,3}[0-9]{1,3}_extension$"
log_ma_extension_features = [f for f in train_X.columns if re.match(regex, f)]
model_dictionary = {}

for log_ma in log_ma_extension_features:
    regex =  r"[A-Z]{2,3}[0-9]{1,3}"
    _match = re.search(regex, log_ma)
    #_data[f"risk_{ma}"] = risk_metric(_data[log_ma])
    model_dictionary[log_ma] = normal_distribution_model(train_X[log_ma])

print(model_dictionary)

# Save models as pickles
import pickle

with open("ETH Risk Models", "wb") as handle:
    pickle.dump(model_dictionary, handle, protocol = pickle.HIGHEST_PROTOCOL)

{'log_MA100_extension': NormalDistributionModel(model=<scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7f980d7c0df0>, mu=0.2474121666138357, var=0.7793789406245433), 'log_MA50_extension': NormalDistributionModel(model=<scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7f980d595990>, mu=0.18783792212198896, var=0.46449107651352606), 'log_EMA21_extension': NormalDistributionModel(model=<scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7f980d7c0310>, mu=0.08536350814886386, var=0.13383488730070298), 'log_MA20_extension': NormalDistributionModel(model=<scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7f980d7c28f0>, mu=0.1132855040708293, var=0.17793296439994105), 'log_MA8_extension': NormalDistributionModel(model=<scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7f980d7fa4d0>, mu=0.04132221387885157, var=0.061541663615807214)}


/home/brad/workspace/autotrader/logic/normal.py:28: RuntimeWarning: divide by zero encountered in log
  - (n / 2) * np.log(var)
/home/brad/workspace/autotrader/logic/normal.py:29: RuntimeWarning: divide by zero encountered in scalar divide
  - (1 / (2 * var) * sum((x - mu) ** 2))
/home/brad/workspace/autotrader/logic/normal.py:27: RuntimeWarning: invalid value encountered in scalar subtract
  -(n / 2) * np.log(2 * np.pi)


## Running New Data Through Existing Models

In [30]:
import pickle

with open("ETH Risk Models", "rb") as handle:
    _model_dictionary = pickle.load(handle)

# get btc weekly data 2020 onwards
_data = read_data("ETH", "1W")
_processed_data = log_close_sma_extension(_data)
test_X = _processed_data.iloc[-1,:].copy(deep=True)

# calculate risk
import numpy as np


for log_ma in log_ma_extension_features:
    regex =  r"[A-Z]{2,3}[0-9]{1,3}"
    _match = re.search(regex, log_ma)
    ma = _match.group(0)
    x = test_X[log_ma]
    pdf_data = _model_dictionary[log_ma].model.pdf(x)
    normalised_pdf_data = pdf_data / _model_dictionary[log_ma].model.pdf(_model_dictionary[log_ma].mu)
    factor = np.empty_like(np.array(x))
    if x > _model_dictionary[log_ma].mu:
        factor = 1
    else:
        factor = -1
    test_X[f"risk_{ma}"] = 0.5 + (factor * (1 - normalised_pdf_data) * 0.5)

test_X["combined_risk"] = (
        (0.5*test_X["risk_EMA21"]) 
        + test_X["risk_MA50"] 
        + (0.5*test_X["risk_MA20"])
        + test_X["risk_MA8"]
    ) / 3

test_X

DateTime               2024-01-29 00:00:00
open                               2256.93
high                               2381.03
low                                2233.92
close                              2369.64
Volume                       181114.252951
Volume MA                    939567.911394
MA100                            1817.3691
MA50                              1898.865
EMA21                          2122.113497
MA20                              2052.383
MA8                             2315.00375
RSI                              61.819428
RSI MA                           64.244169
ATR1                                147.11
ATR14                              260.635
log_MA100_extension               0.265348
log_MA50_extension                0.221482
log_EMA21_extension               0.110326
log_MA20_extension                0.143736
log_MA8_extension                 0.023327
risk_MA100                        0.500132
risk_MA50                          0.50131
risk_EMA21 